<a href="https://colab.research.google.com/github/shionguha/inf2178-expdesignfordatascience-w23/blob/main/notebooks/t_tests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Class***
INF2178 

***Authors***
Yu Wan, Kristina Foster

***Date***
February 28, 2023

# Imports and Preprocessing

In [ ]:
# import libraries
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from statsmodels.stats.proportion import proportions_ztest
from scipy.stats import chi2_contingency
from scipy import stats 
from scipy.stats import ttest_ind
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.graphics.factorplots import interaction_plot
import matplotlib.pyplot as plt
from scipy import stats

#Clean data
def cleandata(filename):
    """
    Takes in csv file name in the format 'file_name.csv'
    Collapses Action at Arrest variables into a list of actions associated with each arrest
    Returns cleaned dataframe 
    """
    event_data = pd.read_csv(filename)
    
    #Remove U type genders because the population size is only 9 and not a good representation of the whole.

    # Checking for duplicate rows
    duplicate_rows_df = event_data[event_data.duplicated()]
    print('Number of duplicate rows: ', duplicate_rows_df.shape)
    
    return event_data

# Experimental Design Analysis

### Attributes and levels for arrest_data

#### Observational units

* Arrest 

#### Occurrence_Category
Represents the categories of the type of occurrence related to the arrest

dataset = Object
* 'Assault & Other crimes against persons'
* 'Harassment/Threatening',
* 'FTA/FTC/Compliance Check/Parollee', 
* 'Assault', 
* 'Robbery/Theft',
* 'Sexual Related Crime', 
* 'Mischief & Fraud', 
* 'Warrant',
* 'Police Category - Administrative', 
* 'Robbery & Theft',
* 'FTA/FTC, Compliance Check & Parollee', 
* 'Drug Related', 
* 'Weapons',
* 'Vehicle Related (inc. Impaired)', 
* 'Other Offence',
* 'Harassment & Threatening', 
* 'Weapons & Homicide',
* 'Break and Enter', 
* 'Mischief', 
* 'Break & Enter',
* 'Sexual Related Crimes & Crimes Against Children',
* 'Crimes against Children', 
* 'Police Category - Incident', 
* 'Fraud',
* 'LLA', 
* 'Mental Health', 
* 'Other Statute', 
* *'Vehicle Related',
* 'Other Statute & Other Incident Type', 
* 'Impaired', 
* 'Homicide', 
* NaN

#### Sex
Represents the sex assigned at birth for the arrestee.

dataType = Object
* F — Female
* M — Male

#### Race
Represents the race that the officers perceived when looking at the arrestee.

dataType = Object
* 'White'
* 'Unknown or Legacy'
* 'Black'
* 'South Asian'
* 'Indigenous',
* 'Middle-Eastern'
* 'Latino', 
* 'East/Southeast Asian'
* NaN

#### StripSearch
Represents if the individual has been strip searched or not.

dataType = Integer
* 0 — Has not been strip-searched
* 1 — Was strip-searched

### Exploring frequency across discrete variables

In [ ]:
#Generate raw dataset
arrest_data = pd.read_csv('inf2178_midterm_data.csv')

#for event in event_arrests_data:
arrest_data = cleandata('inf2178_midterm_data.csv')
print("Number of unique arrests: " + str(arrest_data.shape[0]))

#### Sex <> Occurrence Category

In [ ]:
gender_counts = arrest_data.groupby(['Occurrence_Category', 'Sex'])['Sex'].count().unstack()
print(gender_counts)

In [ ]:
arrest_data = arrest_data[arrest_data['Sex'] != 'U']

#### Sex <> Occurrence Type for Strip Search Population Bar Chart (Figure 3)

In [ ]:
#to see how many people get arrest by male and female in each type Occurrence
gender_counts = arrest_data.groupby(['Occurrence_Category', 'Sex'])['Sex'].count().unstack()

gender = gender_counts.reset_index()
width=0.4
male_amount = gender_counts['M']
female_amount = gender_counts['F']

category= gender['Occurrence_Category']
values=np.arange(len(category))

plt.bar(values,male_amount, color='#4682B4', label='Male',width = 0.4)
plt.bar(values+width,female_amount, color='#DC143C', label='Female', width =0.4)

plt.xticks(values,category,rotation = 90)

plt.xlabel('Occurrence_Category')
plt.ylabel('Number of Arrest')
plt.title('Arrested by Gender and Occurrence_Category')
plt.legend(title='Gender')
plt.grid(False)
plt.show()

#### Occurrence Type <> Strip Searched Bar chart (Figure 4) 

In [ ]:
# count in each category, how many male and female received strip search
strip_data = arrest_data[arrest_data['StripSearch'] == 1]
strip_counts = strip_data.groupby(['Occurrence_Category', 'Sex'])['Sex'].count().unstack()
strip_counts = strip_counts.fillna(0)

# merge the result with Occurrence_Category to create a new 
result4 = strip_counts.merge(gender['Occurrence_Category'], on='Occurrence_Category')
male_strip = strip_counts['M']
female_strip = strip_counts['F']
category1= result4['Occurrence_Category']
width=0.4
values=np.arange(len(category1))

plt.bar(values,male_strip, color='#4682B4', label='Male',width = 0.4)
plt.bar(values+width,female_strip, color='#DC143C', label='Female', width =0.4)

plt.xticks(values,category1,rotation = 90)

plt.xlabel('Occurrence_Category')
plt.ylabel('Number of Strip Search')
plt.title('Strip Search by Gender and Occurrence_Category')
plt.legend(title='Gender')
plt.grid(False)
plt.show()

## Discrete probability distributions by variable

In [ ]:
#Create discrete continuous variable for joint distribution across arrest reasons

#Function to return a PMF of an array of variables
#Source: https://allendowney.github.io/BiteSizeBayes/10_joint.html
def pmf_from_seq(seq):
    """Make a PMF from a sequence of values.
    
    seq: sequence
    
    returns: Series representing a PMF
    """
    pmf = pd.Series(seq).value_counts(sort=False).sort_index()
    pmf /= pmf.sum()
    return pmf

In [ ]:
#PMF for Sex
sex_arrest = pmf_from_seq(arrest_data["Sex"])
sex_arrest.plot.bar(color='C0', alpha=0.5)
plt.xlabel('Sex')
plt.ylabel('Probability')
plt.title('Distribution for Probability of Sex');

In [ ]:
#PMF for Race
race_arrest = pmf_from_seq(arrest_data["Perceived_Race"])
race_arrest.plot.bar(color='C1', alpha=0.5)
plt.xlabel('Race')
plt.ylabel('Probability')
plt.title('Distribution for Probability of Race');

In [ ]:
#PMF for Occurrance Category
occurrance_arrest = pmf_from_seq(arrest_data["Occurrence_Category"])
occurrance_arrest.plot.bar(color='C2', alpha=0.5)
plt.xlabel('Occurrance_Category')
plt.ylabel('Probability')
plt.title('Frequency of the Probability for types of Occurrences');

In [ ]:
#PMF for Strip Searched
stripped_arrest = pmf_from_seq(arrest_data["StripSearch"])
stripped_arrest.plot.bar(color='C3', alpha=0.5)
plt.xlabel('Probability of Strip Search')
plt.ylabel('Probability')
plt.title('Distribution of Probability for Strip Searches');

## Joint Distribution

In [ ]:
#Source: https://allendowney.github.io/BiteSizeBayes/10_joint.html
def plot_heatmap(xtab):
    """Make a heatmap to represent a cross tabulation.
    xtab: DataFrame containing a cross tabulation
    """
    plt.pcolormesh(xtab)

    # label the y axis
    ys = xtab.index
    plt.ylabel(ys.name)
    locs = np.arange(len(ys)) + 0.5
    plt.yticks(locs, ys)

    # label the x axis
    xs = xtab.columns
    plt.xlabel(xs.name)
    locs = np.arange(len(xs)) + 0.5
    plt.xticks(locs, xs)
    
    plt.colorbar()
    plt.gca().invert_yaxis()


### Occurrance Type <> Strip Searches

In [ ]:
#Constructing crosstab for Joint PMF
ct_type_stripsearch = pd.crosstab(index=arrest_data["Occurrence_Category"], columns=arrest_data["StripSearch"], normalize=True)

#Display header of Joint PMF
ct_type_stripsearch.head()

In [ ]:
plot_heatmap(ct_type_stripsearch)

### 1.2 Sex <> Strip Searches

In [ ]:
#Constructing crosstab for Joint PMF
ct_sex_stripsearch = pd.crosstab(index=arrest_data["Sex"], columns=arrest_data["StripSearch"], normalize=True)

#Display header of Joint PMF
ct_sex_stripsearch.head()

In [ ]:
plot_heatmap(ct_sex_stripsearch)

### Perceived Race <> Strip Searches

In [ ]:
#Constructing crosstab for Joint PMF
ct_race_stripsearch = pd.crosstab(index=arrest_data["Perceived_Race"], columns=arrest_data["StripSearch"], normalize=True)

#Display header of Joint PMF
ct_race_stripsearch.head()

In [ ]:
plot_heatmap(ct_race_stripsearch)

### Joint Distribution Visualization

#### Occurrance Type <> Strip Searches

In [ ]:
#Create Occurrence Category joint marginal distributions for occurrence type and strip-searches
stripped = arrest_data[arrest_data['StripSearch'] == 1].copy()
not_stripped = arrest_data[arrest_data["StripSearch"] != 1].copy()

#determine joint marginal distribution for arrests
joint_margin_freq_stripped = stripped['Occurrence_Category'].value_counts(normalize=True)
joint_margin_freq_not_stripped = not_stripped['Occurrence_Category'].value_counts(normalize=True)

#Add columns store joint marginal distributions of frequency Occurence Type
stripped['Occurrence_Category_joint_margin'] = stripped['Occurrence_Category'].map(joint_margin_freq_stripped)
not_stripped['Occurrence_Category_joint_margin2'] = not_stripped['Occurrence_Category'].map(joint_margin_freq_not_stripped)
                                                 
#Concatenate datasets
arrest_data = pd.concat([stripped,not_stripped]).copy()

#Merge both marginal distributions into one column
arrest_data['Occurrence_Category_joint_margin_merged'] = arrest_data[['Occurrence_Category_joint_margin','Occurrence_Category_joint_margin2']].bfill(axis=1).iloc[:, 0]

#Add a categorical scale for occurrence category
arrest_data["Occurrence_Category"] = arrest_data["Occurrence_Category"].astype('category')
arrest_data["Occurrence_Category_Num"] = arrest_data["Occurrence_Category"].cat.codes

arrest_data.head()

In [ ]:
#Compare frequency of an occurrence of being strip searched based off of the occurrence type
sns.catplot(data=arrest_data, kind="box", x="Occurrence_Category_joint_margin_merged", y="Occurrence_Category", hue="StripSearch")
plt.xlabel("Strip Searched")
plt.ylabel("Frequency of the Marginal Distribution for Occurrence Type")
plt.figure(figsize=(15,35))

#### Perceived Race <> Strip Searches

In [ ]:
#Create Occurrence Category joint marginal distributions for race and strip-searches
stripped = arrest_data[arrest_data['StripSearch'] == 1].copy()
not_stripped = arrest_data[arrest_data["StripSearch"] != 1].copy()

#determine joint marginal distribution for arrests
joint_margin_freq_stripped = stripped['Perceived_Race'].value_counts(normalize=True)
joint_margin_freq_not_stripped = not_stripped['Perceived_Race'].value_counts(normalize=True)

#Add columns store joint marginal distributions of frequency race
stripped['Race_joint_margin'] = stripped['Perceived_Race'].map(joint_margin_freq_stripped)
not_stripped['Race_joint_margin2'] = not_stripped['Perceived_Race'].map(joint_margin_freq_not_stripped)
                                                 
#Concatenate datasets
arrest_data = pd.concat([stripped,not_stripped]).copy()

#Merge both marginal distributions into one column
arrest_data['Perceived_Race_joint_margin_merged'] = arrest_data[['Race_joint_margin','Race_joint_margin2']].bfill(axis=1).iloc[:, 0]

#Add a categorical scale for race
arrest_data["Perceived_Race"] = arrest_data["Perceived_Race"].astype('category')
arrest_data["Perceived_Race_Category"] = arrest_data["Perceived_Race"].cat.codes

arrest_data.head()

In [ ]:
#Compare frequency of an occurrence of being strip searched based off of the person's race.
sns.catplot(data=arrest_data, kind="box", x="Perceived_Race_joint_margin_merged", y="Perceived_Race", hue="StripSearch")
plt.xlabel("Strip Searched")
plt.ylabel("Frequency of the Marginal Distribution for Perceived Race")

## T-Tests across variables

### Is there any difference between male and female being strip-searched?

In [ ]:
#probability of male and female being strip searched.
gender_strip_probs = arrest_data.groupby('Sex')['StripSearch'].mean()
print(gender_strip_probs)

In [ ]:
male_arrest = arrest_data[arrest_data["Sex"] == "M"]
female_arrest = arrest_data[arrest_data["Sex"] == "F"]
t_stat, p_val = stats.ttest_ind(male_arrest["StripSearch"], female_arrest["StripSearch"])
print('t_stat:',t_stat)
print('p value:',p_val)

In [ ]:
# There is a significant difference between male and female being strip searched since p_val <0.05

### Is there any difference between male and female being strip-searched if they caught by the reason Break and Enter?

In [ ]:
break_and_enter_data = arrest_data[arrest_data['Occurrence_Category'] == 'Break & Enter']
gender_strip_probs_break = break_and_enter_data.groupby('Sex')['StripSearch'].mean()
print(gender_strip_probs_break)

In [ ]:
male_break_and_enter = break_and_enter_data[break_and_enter_data["Sex"] == "M"]
male_strip_search_prop_break = male_break_and_enter['StripSearch'].mean()
female_break_and_enter = break_and_enter_data[break_and_enter_data["Sex"] == "F"]
female_strip_search_prop_break = female_break_and_enter['StripSearch'].mean()
t_stat, p_val = stats.ttest_ind(male_break_and_enter["StripSearch"], female_break_and_enter["StripSearch"])
print('mean for male being strip searched under break and enter:',male_strip_search_prop_break)
print('mean for female being strip searched under break and enter:',female_strip_search_prop_break)
print('t-stat:',t_stat)
print('p_val:',p_val)

#### Results
No difference since p-val > 0.05

### Is there any difference between male and female being strip-searched if they caught by the reason Drug Related?

In [ ]:
drug_related_data = arrest_data[arrest_data['Occurrence_Category'] == 'Drug Related']
gender_strip_probs_drug = drug_related_data.groupby('Sex')['StripSearch'].mean()
print(gender_strip_probs_drug)

In [ ]:
male_drug_related = drug_related_data[drug_related_data['Sex'] == 'M']
male_strip_search_prop_drug = male_drug_related['StripSearch'].mean()
female_drug_related = drug_related_data[drug_related_data['Sex'] == 'F']
female_strip_search_prop_drug = female_drug_related['StripSearch'].mean()
t_stat, p_val = stats.ttest_ind(male_drug_related['StripSearch'], female_drug_related['StripSearch'])
print('mean for male being strip searched under drug and related:',male_strip_search_prop_drug)
print('mean for female being strip searched under drug and related:',female_strip_search_prop_drug)
print('t_stat:',t_stat)
print('p_val:',p_val)

#### Results
There is a significantly difference, since p-value < 0.05

### Is there any difference between male and female being strip-searched if they caught by the reason of Weapons & Homicide?

In [ ]:
weapons_homicide_data = arrest_data[arrest_data['Occurrence_Category'] == 'Weapons & Homicide']
gender_strip_probs_weapons_homicide = weapons_homicide_data.groupby('Sex')['StripSearch'].mean()
print(gender_strip_probs_weapons_homicide)

In [ ]:
male_weapons_homicide = weapons_homicide_data[weapons_homicide_data["Sex"] == "M"]
male_weapons_homicide_prop = male_weapons_homicide['StripSearch'].mean()
female_weapons_homicide = weapons_homicide_data[weapons_homicide_data["Sex"] == "F"]
female_weapons_homicide_prop = female_weapons_homicide['StripSearch'].mean()

t_stat, p_val = stats.ttest_ind(male_weapons_homicide["StripSearch"], female_weapons_homicide["StripSearch"])
print('mean for male being strip searched under weapons and homicide:',male_weapons_homicide_prop)
print('mean for female being strip searched nder weapons and homicide:',female_weapons_homicide_prop)
print('t_stat:',t_stat)
print('p_val:',p_val)

#### Results
No difference since p-val > 0.05

### Is there any difference between the mean of indigenous and all other races strip-searched 

In [ ]:
# Divide the data into two subset that include only Indigenous and the rest of the races
indigenous = arrest_data[arrest_data['Perceived_Race'] == 'Indigenous']
rest_race = arrest_data[arrest_data['Perceived_Race'] != 'Indigenous']

# Caculate the mean  for each subset
indigenous_prob1 = indigenous['StripSearch'].mean()
rest_race_prob1 = rest_race['StripSearch'].mean()

# Calculate the t-test and get p-value
t_stat, p_value = ttest_ind(indigenous['StripSearch'], rest_race['StripSearch'], equal_var=False)

print('mean of indigenous people:',indigenous_prob1)
print('mean of rest race:',rest_race_prob1)
print('t_stat:',t_stat)
print('p_val:',p_value)

#### Results
There is a significantly difference, since p-value < 0.05

## Two-Way ANOVA: Perceived Race <> Sex

In [ ]:
#Two-Way ANOVA
from statsmodels.formula.api import ols
import statsmodels.api as sm
# Specify the model formula
model = ols('Perceived_Race_joint_margin_merged~Perceived_Race + Sex + Sex:Perceived_Race', data=arrest_data)

# Fit the model
results = model.fit()

# Print the ANOVA table
print(sm.stats.anova_lm(results, typ=2))

In [ ]:
#Interaction for Joint Marginal Race <> StripSearch
strip_search = arrest_data["StripSearch"].astype("category")
joint_margin_occur_freq = arrest_data["Perceived_Race_joint_margin_merged"].values

interaction_plot_freq = interaction_plot(arrest_data["Perceived_Race_Category"], strip_search, joint_margin_occur_freq, ms=10)
plt.ylabel('Mean Frequencies for Joint Marginal Distribution')
plt.xlabel('Race Category')
plt.title('Joint marginal mean frequency of race separated by strip search', fontweight='bold')
plt.show()